In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

# =====================================================
# 1️⃣ 데이터 로드 + 시간 파생
# =====================================================
df = pd.read_csv("new_flight_weather_merged.csv")

df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])
df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

# =====================================================
# 2️⃣ 문제없음 vs 취소만 필터링
# =====================================================
df = df[df["상태"].isin(["정상운항", "취소"])].copy()

df["target"] = (df["상태"] == "취소").astype(int)

print("📊 원본 클래스 분포")
print(df["target"].value_counts())
# 0 = 문제없음, 1 = 취소

# =====================================================
# 3️⃣ 컬럼 정의
# =====================================================
num_cols = ["기온(°C)", "풍속_ms", "dep_hour", "dep_weekday", "is_weekend"]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = ["항공사", "출발지", "도착지", "flight_type"]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype("category")

X_cols = num_cols + cat_cols

# =====================================================
# 4️⃣ Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))

# =====================================================
# 5️⃣ 🔥 취소 기준 다운사이징 (Train만)
# =====================================================
train_ok = train_df[train_df["target"] == 0]
train_cancel = train_df[train_df["target"] == 1]

base_n = len(train_cancel)

print("취소 샘플 수:", base_n)

train_ok_down = train_ok.sample(n=base_n, random_state=42)

train_down = pd.concat([
    train_ok_down,
    train_cancel
]).sample(frac=1, random_state=42)

print("\n📊 다운사이징 후 Train 분포")
print(train_down["target"].value_counts())

# =====================================================
# 6️⃣ X / y 분리
# =====================================================
X_train = train_down[X_cols]
y_train = train_down["target"]

X_test  = test_df[X_cols]
y_test  = test_df["target"]

# =====================================================
# 7️⃣ LightGBM 이진분류
# =====================================================
lgbm = LGBMClassifier(
    objective="binary",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgbm.fit(
    X_train,
    y_train,
    categorical_feature=cat_cols
)

print("\n✅ LightGBM (문제없음 vs 취소) 학습 완료")

# =====================================================
# 8️⃣ 평가
# =====================================================
y_prob = lgbm.predict_proba(X_test)[:, 1]

for t in [0.3, 0.4, 0.5]:
    print(f"\n===== Threshold = {t} =====")
    y_pred = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


C:\Users\Admin\AppData\Local\Temp\ipykernel_14648\524342269.py:9: DtypeWarning: Columns (28,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_flight_weather_merged.csv")


📊 원본 클래스 분포
0    2360824
1        579
Name: target, dtype: int64
Train: 1889123 Test: 472280
취소 샘플 수: 445

📊 다운사이징 후 Train 분포
0    445
1    445
Name: target, dtype: int64
[LightGBM] [Info] Number of positive: 445, number of negative: 445
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000101 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 421
[LightGBM] [Info] Number of data points in the train set: 890, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f


✅ LightGBM (문제없음 vs 취소) 학습 완료

===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       1.00      0.82      0.90    472146
           1       0.00      0.67      0.00       134

    accuracy                           0.82    472280
   macro avg       0.50      0.75      0.45    472280
weighted avg       1.00      0.82      0.90    472280


===== Threshold = 0.4 =====
              precision    recall  f1-score   support

           0       1.00      0.84      0.91    472146
           1       0.00      0.65      0.00       134

    accuracy                           0.84    472280
   macro avg       0.50      0.74      0.46    472280
weighted avg       1.00      0.84      0.91    472280


===== Threshold = 0.5 =====
              precision    recall  f1-score   support

           0       1.00      0.85      0.92    472146
           1       0.00      0.63      0.00       134

    accuracy                           0.85    472280
   macro a